In [5]:
import pandas as pd
df = pd.read_csv("train_clickstreams.tab", sep='\t', encoding="CP949")
#시간 7개 na 제거

In [3]:
df = df.dropna()

In [6]:
df

,CUS_ID,TIME_ID,SITE,SITE_CNT,ST_TIME,SITE_NM,BACT_NM,MACT_NM,ACT_NM
0,1,2012070905,search.naver.com,3.0,794.0,네이버 검색,인터넷/컴퓨터,검색,포털검색
1,1,2012072507,plus.google.com,1.0,1.0,구글 Plus,커뮤니티,블로그/SNS,SNS
2,1,2012081116,joongang.joinsmsn.com,2.0,5.0,중앙일보,뉴스/미디어,일간지,종합일간지
3,1,2012090304,news.naver.com,5.0,504.0,네이버 뉴스,뉴스/미디어,인터넷신문,포털뉴스
4,1,2012090506,news.nate.com,1.0,0.0,네이트 뉴스,뉴스/미디어,인터넷신문,포털뉴스
...,...,...,...,...,...,...,...,...,...
8440576,998,2013020610,www.player.co.kr,3.0,65.0,플레이어,쇼핑,패션잡화 쇼핑몰,종합신발쇼핑몰
8440577,998,2013021315,www.newspim.com,2.0,0.0,뉴스핌,뉴스/미디어,일간지,경제신문
8440578,998,2013022713,www.daum.net,8.0,0.0,다음,인터넷/컴퓨터,포털,종합포털
8440579,999,2012071017,www.kdisk.co.kr,7.0,194.0,케이디스크,인터넷/컴퓨터,다운로드,컨텐츠공유(P2P)


In [2]:
WEEKDAY_DICT = {
    0: 'MON',
    1: 'TUE',
    2: 'WED',
    3: 'THU',
    4: 'FRI',
    5: 'SAT',
    6: 'SUN',
}

MONTH_DICT = {
    1:'Jan', 
    2:'Feb', 
    3:'Mar', 
    4:'Apr', 
    5:'May', 
    6:'Jun', 
    7:'Jul',
    8:'Aug', 
    9:'Sep', 
    10:'Oct', 
    11:'Nov', 
    12:'Dec'
}

def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    """Preprocess data frame for BMA"""
    df = preprocess_time_column(df)
    df = construct_qtile(df,10)
    # 각 아이디별 접속횟수기반
    df1 = aggregate_and_get_pivot(df, 'SITE_CNT', ['CUS_ID'], agg_func = 'mean')
    df2 = aggregate_and_get_pivot(df, 'SITE_CNT', ['CUS_ID'], agg_func = 'sum')
    df3 = aggregate_and_get_pivot(df, 'SITE_CNT', ['CUS_ID', 'month'], agg_func = 'mean')
    df4 = aggregate_and_get_pivot(df, 'SITE_CNT', ['CUS_ID', 'month'], agg_func = 'sum')
    df5 = aggregate_and_get_pivot(df, 'SITE_CNT', ['CUS_ID', 'weekday'], agg_func = 'mean')
    df6 = aggregate_and_get_pivot(df, 'SITE_CNT', ['CUS_ID', 'weekday'], agg_func = 'sum')
    # 각 아이디별 체류시간 기반
    df7 = aggregate_and_get_pivot(df, 'ST_TIME', ['CUS_ID'], agg_func = 'mean')
    df8 = aggregate_and_get_pivot(df, 'ST_TIME', ['CUS_ID'], agg_func = 'sum')
    df9 = aggregate_and_get_pivot(df, 'ST_TIME', ['CUS_ID', 'month'], agg_func = 'mean')
    df10 = aggregate_and_get_pivot(df, 'ST_TIME', ['CUS_ID', 'month'], agg_func = 'sum')
    df11 = aggregate_and_get_pivot(df, 'ST_TIME', ['CUS_ID', 'weekday'], agg_func = 'mean')
    df12 = aggregate_and_get_pivot(df, 'ST_TIME', ['CUS_ID', 'weekday'], agg_func = 'sum')
    # aggregate
    dfs = [df1, df2, df3, df4, df5,df6, df7, df8, df9, df10, df11, df12]
    df = dfs[0].join(dfs[1:], how="outer")
    # fill NaN with 0 
    df.fillna(0)
    
    return df



In [6]:
df = preprocess(df)
df

,SITE_CNT_mean,SITE_CNT_sum,SITE_CNT_mean_Apr,SITE_CNT_mean_Aug,SITE_CNT_mean_Dec,SITE_CNT_mean_Feb,SITE_CNT_mean_Jan,SITE_CNT_mean_Jul,SITE_CNT_mean_Jun,SITE_CNT_mean_Mar,...,ST_TIME_mean_THU,ST_TIME_mean_TUE,ST_TIME_mean_WED,ST_TIME_sum_FRI,ST_TIME_sum_MON,ST_TIME_sum_SAT,ST_TIME_sum_SUN,ST_TIME_sum_THU,ST_TIME_sum_TUE,ST_TIME_sum_WED
CUS_ID,,,,,,,,,,,,,,,,,,,,,
1,3.569263,13656.0,1.250000,3.670909,1.866667,NaN,1.590000,3.450754,3.941624,2.315789,...,105.467509,112.288927,87.624260,46931.0,53595.0,28850.0,46036.0,58429.0,64903.0,59234.0
2,4.711382,25498.0,4.882091,4.496788,3.695238,3.801484,3.873494,7.398625,5.414073,4.179612,...,159.266772,155.449025,149.349827,146984.0,144070.0,6049.0,NaN,201791.0,207369.0,129785.0
3,4.661146,10413.0,NaN,5.232919,4.297297,4.483333,5.779661,4.848039,NaN,NaN,...,206.002494,147.273642,190.852321,64711.0,97761.0,10159.0,14713.0,82607.0,73195.0,45232.0
4,6.523132,12831.0,NaN,5.576786,2.078431,NaN,2.318182,5.885400,NaN,NaN,...,124.625786,110.494220,187.597938,43157.0,56613.0,14876.0,10276.0,39631.0,38231.0,72788.0
5,3.180010,13267.0,4.333333,4.464630,2.741339,2.670157,3.416918,3.011725,NaN,2.325269,...,120.117881,114.531707,113.652397,74568.0,119703.0,66084.0,14068.0,90689.0,70437.0,66373.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2496,4.802320,10349.0,4.323529,NaN,6.801802,5.297994,5.331646,NaN,3.476667,5.806630,...,225.403766,212.525180,210.058957,92772.0,88350.0,26395.0,20209.0,107743.0,59082.0,92636.0
2497,4.403243,8419.0,4.025974,NaN,3.451883,5.528024,4.309804,NaN,4.850427,5.429167,...,117.161290,79.470356,72.115385,18429.0,14189.0,19741.0,20848.0,32688.0,20106.0,20625.0
2498,5.213238,24179.0,4.093385,5.013423,4.662300,5.937743,5.281609,5.928793,4.621302,5.868526,...,122.226984,157.706202,111.059633,67178.0,125006.0,36218.0,83740.0,77003.0,144932.0,72633.0


In [3]:
# 시간 전처리
from datetime import datetime

def preprocess_time_column(df: pd.DataFrame)-> pd.DataFrame:
    """
    step 1. (int)time to datetime
    step 2. make weekday column
    step 3. make month column
    """
#     def _insert_space(df, integer): # for step1
#         return df[0:integer] + ' ' + df[integer:]
    
#     df["TIME_ID"] = df["TIME_ID"].astype(str).apply(lambda x: _insert_space(x,8))
#     df["TIME_ID"] = pd.to_datetime(df["TIME_ID"], infer_datetime_format=True)

    df['TIME_ID'] = df['TIME_ID'].astype(str).apply(lambda x: datetime.strptime(x, '%Y%m%d%H'))
    
    df["weekday"]= df["TIME_ID"].dt\
                                .weekday\
                                .apply(lambda x: WEEKDAY_DICT[x])
    
    df["month"] = df["TIME_ID"].dt\
                               .month\
                               .apply(lambda x: MONTH_DICT[x])
    
    return df


In [4]:
# 주요 사이트 방문횟수로 10분위
def construct_qtile(df, nbins, group_col = 'SITE', agg_col = 'SITE_CNT'):
    """
    construct quantile based on SITE_CNT
    """
    labels = range(nbins, 0, -1)

    ex = df.groupby([group_col])[[agg_col]].sum().reset_index()
    
    ex[f"QTILE_{nbins}"] = pd.qcut(ex[agg_col], nbins + 2,
                          labels=labels, duplicates='drop')
    
    ex = ex.drop(columns = [agg_col])
    
    return pd.merge(df, ex, on=group_col)

#df = construct_qtile(df, 10)

In [5]:
def aggregate_and_get_pivot(df, target_column, group_columns, agg_func = 'mean'):
    """
    Transform panel data into cross-sectional data
    """
    if len(group_columns) == 1:
        df_gp = df.groupby(group_columns)[target_column]
        new_target_column = f"{target_column}_{agg_func}"
        
        if agg_func == 'mean':
            objection = df_gp.mean()
        elif agg_func == 'sum':
            objection = df_gp.sum()
        else:
            raise ValueError("Wrong")
        objection = objection.reset_index()\
                             .rename(columns = {target_column: new_target_column})
        return objection.set_index('CUS_ID')
    
    elif len(group_columns) == 2:
        df_gp = df.groupby(group_columns)[[target_column]]
        new_target_column = f"{target_column}_{agg_func}"
    
        if agg_func == 'mean':
            objection = df_gp.mean()
        elif agg_func == 'sum':
            objection = df_gp.sum()
        else:
            raise ValueError("Wrong")
    
        objection = objection.reset_index()\
                             .rename(columns = {target_column: new_target_column})
    
        pivot_table = pd.pivot_table(objection,
                                     index = group_columns[0],
                                     columns = [group_columns[1]],
                                     values = [new_target_column])[new_target_column]
    
        pivot_table.columns = [f"{new_target_column}_{value}" for value in pivot_table.columns]
            
        return pivot_table
    
    else:
        raise ValueError("Wrong input")


In [ ]:
# 각 아이디별 접속횟수기반
df1 = aggregate_and_get_pivot(df, 'SITE_CNT', ['CUS_ID'], agg_func = 'mean')
df2 = aggregate_and_get_pivot(df, 'SITE_CNT', ['CUS_ID'], agg_func = 'sum')
df3 = aggregate_and_get_pivot(df, 'SITE_CNT', ['CUS_ID', 'month'], agg_func = 'mean')
df4 = aggregate_and_get_pivot(df, 'SITE_CNT', ['CUS_ID', 'month'], agg_func = 'sum')
df5 = aggregate_and_get_pivot(df, 'SITE_CNT', ['CUS_ID', 'weekday'], agg_func = 'mean')
df6 = aggregate_and_get_pivot(df, 'SITE_CNT', ['CUS_ID', 'weekday'], agg_func = 'sum')
# 각 아이디별 체류시간 기반
df7 = aggregate_and_get_pivot(df, 'ST_TIME', ['CUS_ID'], agg_func = 'mean')
df8 = aggregate_and_get_pivot(df, 'ST_TIME', ['CUS_ID'], agg_func = 'sum')
df9 = aggregate_and_get_pivot(df, 'ST_TIME', ['CUS_ID', 'month'], agg_func = 'mean')
df10 = aggregate_and_get_pivot(df, 'ST_TIME', ['CUS_ID', 'month'], agg_func = 'sum')
df11 = aggregate_and_get_pivot(df, 'ST_TIME', ['CUS_ID', 'weekday'], agg_func = 'mean')
df12 = aggregate_and_get_pivot(df, 'ST_TIME', ['CUS_ID', 'weekday'], agg_func = 'sum')
# 합치기
dfs = [df1, df2, df3, df4, df5,df6, df7, df8, df9, df10, df11, df12]
df = dfs[0].join(dfs[1:], how="outer")